# Walking through the nitty gritty: nDCG calculations

In [11]:
import numpy as np

## nDCG Calculations with worked example

In a given testset, some user (let's say User 1) has ten different ratings. When calculating nDCG, we don't actually care what movie they are for; we just care about the numerical values of the ratings. In this case, we'll make up some ratings for a user that we can use as a worked example.

In [12]:
ratings_in_testset = [3, 4, 5, 1, 2, 3, 4, 5, 5, 4, ]

For these 10 items, the RecSys will estimate a rating, presumably with some error. In this worked example, let's assume the error is an alternating plus or minus 0.5.


In [13]:
def fake_estimations(ratings):
    estimated_vals = []
    flip = True
    for rating in ratings:
        if flip:
            estimated_vals.append(rating - 0.5)
        else:
            estimated_vals.append(rating + 0.5)
        flip = not flip
    return estimated_vals
        
estimated_vals = fake_estimations(ratings_in_testset)

Currently, the function that calculates precision, recall, and nDCG expects a list of tuples, with each tuple being a pair of (estimated value, true value)

In [14]:
user_ratings = [(x, y) for x, y in zip(estimated_vals, ratings_in_testset)]

Here's what the list of tuples looks like:

In [15]:
print(user_ratings)

[(2.5, 3), (4.5, 4), (4.5, 5), (1.5, 1), (1.5, 2), (3.5, 3), (3.5, 4), (5.5, 5), (4.5, 5), (4.5, 4)]


Now we'll need to (1) sort the ratings by estimated value and (2) sort the ratings by true value.

In [16]:
# Now here's the calculations
# Sort user ratings by estimated value
user_ratings_sorted_by_est = sorted(user_ratings, key=lambda x: x[0], reverse=True)
user_ratings_sorted_by_true = sorted(user_ratings, key=lambda x: x[1], reverse=True)

In [17]:
print(user_ratings_sorted_by_est)
print(user_ratings_sorted_by_true)

[(5.5, 5), (4.5, 4), (4.5, 5), (4.5, 5), (4.5, 4), (3.5, 3), (3.5, 4), (2.5, 3), (1.5, 1), (1.5, 2)]
[(4.5, 5), (5.5, 5), (4.5, 5), (4.5, 4), (3.5, 4), (4.5, 4), (2.5, 3), (3.5, 3), (1.5, 2), (1.5, 1)]


As we can see from the above, because of the error in our RecSys, the user ratings sorted by estimated value do not match the user ratings sorted by true value. Therefore, we expect to see an nDCG below 1!

We're going to need to define a function that calculates DCG for a given list of ratings. Let's use the formula defined in this paper from MSR: https://dl.acm.org/citation.cfm?doid=1102351.1102363

The numerator is (2^relevance_score - 1) in this definition (others just use relevance_score as the definition).

In [18]:
def dcg_at_k(ratings):
    """
    Discounted cumulative gain at k
    https://en.wikipedia.org/wiki/Discounted_cumulative_gain
    Using formula from this MSR IR paper:
    https://dl.acm.org/citation.cfm?doid=1102351.1102363

    k is assumed to be the length of the input list
    args:
        ratings: a list of relevance scores, e.g. explicit ratings 1-5
    returns:
        a dcg_at_k value
    """
    k = len(ratings)

    return sum([
        (2 ** rating - 1) / 
        (np.math.log(i + 1, 2))
        for rating, i in zip(ratings, range(1, k+1))
    ])

We can use the ratings sorted by true values to calculate ideal nDCG for various k values. In this example, let's just do 10 and 5.

We'll want to get the first k *true ratings* from the list sorted by true ratings as well as the list sorted by estimated ratings, because the nDCG is calculated based on the *true ratings* and not the estimated ratings.

Since we've put our data in tuples of (estimated, true), we can get the true value by accessing index 1.

In [19]:
true_ratings_of_first_10_true = [x[1] for x in user_ratings_sorted_by_true[:10]]
true_ratings_of_first_10_est = [x[1] for x in user_ratings_sorted_by_est[:10]]

In [20]:
true_ratings_of_first_5_true = [x[1] for x in user_ratings_sorted_by_true[:5]]
true_ratings_of_first_5_est = [x[1] for x in user_ratings_sorted_by_est[:5]]

In [21]:
ideal_dcg_at_10 = dcg_at_k(true_ratings_of_first_10_true)
ideal_dcg_at_5 = dcg_at_k(true_ratings_of_first_5_true)
print('At 10:', ideal_dcg_at_10, 'At 5:', ideal_dcg_at_5)

At 10: 89.3986129310978 At 5: 78.3217628403342


Now calculate the dcg based on estimated values

In [22]:
est_dcg_at_10 = dcg_at_k(true_ratings_of_first_10_est)
est_dcg_at_5 = dcg_at_k(true_ratings_of_first_5_est)
print('At 10:', est_dcg_at_10, 'At 5:', est_dcg_at_5)

At 10: 85.98764063423907 At 5: 75.11771171236516


And finally, we can add the n to nDCG by normalizing!

In [23]:
ndcg_at_10 = est_dcg_at_10 / ideal_dcg_at_10
ndcg_at_5 = est_dcg_at_5 / ideal_dcg_at_5
print('nDCG@10:', ndcg_at_10, 'nDCG@5:', ndcg_at_5)

nDCG@10: 0.9618453554812123 nDCG@5: 0.9590911770652969


## Some problems that might arise...

What if a user doesn't have ten ratings in a testset? How do we compute nDCG@10 for that testset?

In [24]:
small_list_of_ratings = [1, 3, 5]

small_user_ratings = [(x, y) for x, y in zip(fake_estimations(small_list_of_ratings), small_list_of_ratings)]
print(small_user_ratings)

[(0.5, 1), (3.5, 3), (4.5, 5)]
